# SQL_ONE - Análise de Dados com SQL
## Aula 2 - Join, Union

### 1 - Tabela pilotos_cidades
#### Acessando a tabela pilotos_cidades criada no notebook anterior.

In [0]:
-- 1 - Visualização da tabela pilotos_cidades
SELECT * FROM pilotos_cidades;

ID,NOME,IDADE,NATURALIDADE
1,Ayrton Senna,62,São Paulo
2,Rubens Barrichello,50,São Paulo
3,Lewis Hamilton,37,Stevenage
4,Michael Schumacher,53,Hürth
5,Max Verstappen,25,Hasselt


### 2 - Limpando o diretório
#### Caso dê erro, teremos que limpar o diretório e refazer o processo de criação da tabela novamente, procedimento visto no notebook anterior.

In [0]:
-- %fs rm -r dbfs:/user/hive/warehouse/pilotos_cidades
-- ou
-- dbutils.fs.rm("dbfs:/user/hive/warehouse/pilotos_cidades", recurse = True)

In [0]:
/* 2 - Criando a estrutura da base de dados com drop table e create table */

DROP TABLE IF EXISTS pilotos_cidades;

CREATE TABLE pilotos_cidades
(
  ID INT,
  NOME VARCHAR(20),
  IDADE INT,
  NATURALIDADE VARCHAR(30)
);

INSERT INTO
pilotos_cidades
(ID, NOME, IDADE, NATURALIDADE)
VALUES
(1, "Ayrton Senna", 62, "São Paulo"),
(2, "Rubens Barrichello", 50, "São Paulo"),
(3, "Lewis Hamilton", 37, "Stevenage"),
(4, "Michael Schumacher", 53, "Hürth"),
(5, "Max Verstappen", 25, "Hasselt");

SELECT * FROM pilotos_cidades;

ID,NOME,IDADE,NATURALIDADE
1,Ayrton Senna,62,São Paulo
2,Rubens Barrichello,50,São Paulo
3,Lewis Hamilton,37,Stevenage
4,Michael Schumacher,53,Hürth
5,Max Verstappen,25,Hasselt


### 3 - Pandas, GitHub e Spark
#### Outra possibilidade é o acesso a arquivos na web, para isso utilizaremos o PANDAS e salvaremos a base lida com o SPARK.

In [0]:
%py

# 3 - Importando o pandas
import pandas as pd

# Atribuindo a uma variável a URL do arquivo CSV no GitHub
csv_url = "https://raw.githubusercontent.com/wallacecarlis/arquivos/main/pilotos_equipes.csv"

# Lendo o arquivo CSV diretamente do URL
df = pd.read_csv(csv_url)

# Criando um dataframe do spark
spark_df = spark.createDataFrame(df)

# Salvando o DataFrame como uma tabela Delta
spark_df.write.format("delta").mode("overwrite").saveAsTable("pilotos_equipes")

### 4 - Select
#### Uma vez que acessamos a base com o PANDAS e a escrevemos com o SPARK, salvando como uma TABLE, vamos visualizar com o SELECT.

In [0]:
-- 4 - Visualizando a tabela pilotos_equipes
SELECT * FROM pilotos_equipes;

id,nome,equipe
1,Ayrton Senna,Toleman
6,Nelson Piquet,Ensign
3,Lewis Hamilton,McLaren
8,Emerson Fittipaldi,Lotus
9,José Carlos Pace,Willys
2,Rubens Barrichello,Jordan


### 5 - Join
#### Utilizamos o JOIN para juntar as bases a partir de uma coluna em comum, no caso a coluna de ID. Podemos também nomear ou apelidar as bases para facilitar nossa escrita.

In [0]:
-- 5 - Realizando um inner join
SELECT 
  C.ID,
  C.NOME,
  C.IDADE,
  C.NATURALIDADE,
  E.EQUIPE 
FROM pilotos_cidades C 
INNER JOIN pilotos_equipes E
ON C.ID = E.ID

ID,NOME,IDADE,NATURALIDADE,EQUIPE
1,Ayrton Senna,62,São Paulo,Toleman
3,Lewis Hamilton,37,Stevenage,McLaren
2,Rubens Barrichello,50,São Paulo,Jordan


### 6 - Left Join
#### Quando quisermos dar preferência aos dados da tabela da esquerda, utilizamos o LEFT JOIN. A consulta retornará todos os dados da tabela da esquerda, e os dados que não existirem na tabela da direita serão preenchidos com valores nulos (NULL).

In [0]:
-- 6 - Realizando um left join
SELECT 
  C.ID,
  C.NOME,
  C.IDADE,
  C.NATURALIDADE,
  E.EQUIPE 
FROM pilotos_cidades C 
LEFT JOIN pilotos_equipes E
ON C.ID = E.ID

ID,NOME,IDADE,NATURALIDADE,EQUIPE
1,Ayrton Senna,62,São Paulo,Toleman
2,Rubens Barrichello,50,São Paulo,Jordan
3,Lewis Hamilton,37,Stevenage,McLaren
4,Michael Schumacher,53,Hürth,null
5,Max Verstappen,25,Hasselt,null


### 7 - Right Join
#### Quando quisermos dar preferência aos dados da tabela da direita, utilizamos o RIGHT JOIN. A consulta retornará todos os dados da tabela da direita, e os dados que não existirem na tabela da esquerda serão preenchidos com valores nulos (NULL).


In [0]:
-- 7 - Realizando um right join
SELECT 
  E.ID,
  E.NOME,
  C.IDADE,
  C.NATURALIDADE,
  E.EQUIPE 
FROM pilotos_cidades C 
RIGHT JOIN pilotos_equipes E
ON C.ID = E.ID

ID,NOME,IDADE,NATURALIDADE,EQUIPE
1,Ayrton Senna,62,São Paulo,Toleman
6,Nelson Piquet,null,null,Ensign
3,Lewis Hamilton,37,Stevenage,McLaren
8,Emerson Fittipaldi,null,null,Lotus
9,José Carlos Pace,null,null,Willys
2,Rubens Barrichello,50,São Paulo,Jordan


### 8 - Union
#### Quando quisermos simplesmente unir nossas bases utilizaremos o UNION, e caso as bases tenham diferentes quantidades de colunas, utilizaremos o UNION ALL, preenchendo com valores nulos onde não houver correspondente.

In [0]:
-- 8 - Visualizando a base pilotos_cidades
SELECT * FROM pilotos_cidades;

ID,NOME,IDADE,NATURALIDADE
1,Ayrton Senna,62,São Paulo
2,Rubens Barrichello,50,São Paulo
3,Lewis Hamilton,37,Stevenage
4,Michael Schumacher,53,Hürth
5,Max Verstappen,25,Hasselt


In [0]:
-- 9 - Visualizando a base pilotos_equipes
SELECT * FROM pilotos_equipes;

id,nome,equipe
1,Ayrton Senna,Toleman
6,Nelson Piquet,Ensign
3,Lewis Hamilton,McLaren
8,Emerson Fittipaldi,Lotus
9,José Carlos Pace,Willys
2,Rubens Barrichello,Jordan


In [0]:
-- 10 - Unindo as bases no sql com union all
SELECT 
  id, 
  nome, 
  idade, 
  naturalidade,  
  null as equipe
FROM pilotos_cidades 
UNION ALL
SELECT 
  id, 
  nome, 
  null as idade, 
  null as naturalidade, 
  equipe 
FROM pilotos_equipes

id,nome,idade,naturalidade,equipe
1,Ayrton Senna,62,São Paulo,null
2,Rubens Barrichello,50,São Paulo,null
3,Lewis Hamilton,37,Stevenage,null
4,Michael Schumacher,53,Hürth,null
5,Max Verstappen,25,Hasselt,null
1,Ayrton Senna,null,null,Toleman
6,Nelson Piquet,null,null,Ensign
3,Lewis Hamilton,null,null,McLaren
8,Emerson Fittipaldi,null,null,Lotus
9,José Carlos Pace,null,null,Willys
